# NOTE: Main Changes of Original Model
## Try Different Parameters in Each Model for Multiple Times (Detailed Outputs in the Excel Files - Post On BB)
## 2 Add a XGBoost At the End of This File


# Modeling

In [1]:
from datetime import datetime
start_time = datetime.now()

# Libraries to load
# !pip install lightgbm

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import numpy as np
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:02.354226


In [2]:
%%time
vars_total = pd.read_csv('vars_total.csv')

CPU times: user 1min 22s, sys: 32 s, total: 1min 54s
Wall time: 1min 59s


In [3]:
vars = vars_total[['record',
                   'fulladdress_count_30',
                   'ssn_firstname_count_30',
                   'homephone_count_3',
                   'ssn_dob_count_30',
                   'fulladdress_homephone_count_30',
                   'fulladdress_count_0_by_30',
                    'fulladdress_unique_count_for_dob_homephone_60',
                    'fulladdress_unique_count_for_ssn_name_60',
                    'fulladdress_unique_count_for_ssn_lastname_60',
                    'fulladdress_unique_count_for_ssn_60',
                    'fulladdress_unique_count_for_ssn_fulladdress_60',
                    'fulladdress_unique_count_for_ssn_zip5_60',
                    'fulladdress_unique_count_for_name_dob_60',
                    'fulladdress_unique_count_for_fulladdress_dob_60',
                   'fulladdress_unique_count_for_ssn_name_30',
                    'fulladdress_unique_count_for_ssn_lastname_30',
                    'fulladdress_unique_count_for_ssn_fulladdress_30',
                    'fulladdress_unique_count_for_ssn_30',
                    'fulladdress_unique_count_for_ssn_zip5_30',
                    'fulladdress_unique_count_for_ssn_name_dob_30',
                    'fulladdress_count_0_by_14',
                    'fulladdress_unique_count_for_name_fulladdress_30',
                    'fulladdress_unique_count_for_name_dob_30',
                     'name_dob_count_30',
                    'ssn_name_dob_count_30',
                   'fraud_label']]

In [4]:
# use this to cap variables. For some problems it helps
Clip = 10

In [5]:
vars.rename(columns={'fraud_label':'Fraud'},inplace=True)

/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
vars['Fraud'].sum()

14393

In [7]:
vars.head(10)

,record,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,fulladdress_count_0_by_30,fulladdress_unique_count_for_dob_homephone_60,fulladdress_unique_count_for_ssn_name_60,fulladdress_unique_count_for_ssn_lastname_60,...,fulladdress_unique_count_for_ssn_fulladdress_30,fulladdress_unique_count_for_ssn_30,fulladdress_unique_count_for_ssn_zip5_30,fulladdress_unique_count_for_ssn_name_dob_30,fulladdress_count_0_by_14,fulladdress_unique_count_for_name_fulladdress_30,fulladdress_unique_count_for_name_dob_30,name_dob_count_30,ssn_name_dob_count_30,Fraud
0,1,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
1,2,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,1
2,3,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
3,4,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
4,5,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
5,6,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
6,7,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
7,8,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
8,9,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0
9,10,1,1,1,1,1,30.0,1,1,1,...,1,1,1,1,14.0,1,1,1,1,0


In [8]:
vars.shape

(1000000, 27)

In [9]:
vars.describe()

,record,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,fulladdress_count_0_by_30,fulladdress_unique_count_for_dob_homephone_60,fulladdress_unique_count_for_ssn_name_60,fulladdress_unique_count_for_ssn_lastname_60,...,fulladdress_unique_count_for_ssn_fulladdress_30,fulladdress_unique_count_for_ssn_30,fulladdress_unique_count_for_ssn_zip5_30,fulladdress_unique_count_for_ssn_name_dob_30,fulladdress_count_0_by_14,fulladdress_unique_count_for_name_fulladdress_30,fulladdress_unique_count_for_name_dob_30,name_dob_count_30,ssn_name_dob_count_30,Fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,500000.500000,1.064624,1.048903,1.449296,1.046097,1.048207,29.491030,1.044485,1.038698,1.038636,...,1.036240,1.036240,1.036240,1.038982,13.866195,1.035916,1.038716,1.046295,1.045940,0.014393
std,288675.278933,0.633831,0.499029,0.861622,0.496315,0.507682,2.831175,0.617953,0.613817,0.613757,...,0.608032,0.608032,0.608032,0.610242,1.015231,0.607661,0.609942,0.496914,0.496147,0.119104
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.304348,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.636364,1.000000,1.000000,1.000000,1.000000,0.000000
25%,250000.750000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,500000.500000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,0.000000
75%,750000.250000,1.000000,1.000000,2.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1000000.000000,30.000000,34.000000,32.000000,34.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,14.000000,30.000000,30.000000,34.000000,34.000000,1.000000


In [10]:
record_save = vars['record']
Y_save = pd.DataFrame(vars.loc[:,'Fraud'])
Y_save.head()

,Fraud
0,0
1,1
2,0
3,0
4,0


## Scale and truncate field values

In [11]:
X_no_scaling = vars.drop(columns = ['record','Fraud'])
X_no_scaling.describe()

,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,fulladdress_count_0_by_30,fulladdress_unique_count_for_dob_homephone_60,fulladdress_unique_count_for_ssn_name_60,fulladdress_unique_count_for_ssn_lastname_60,fulladdress_unique_count_for_ssn_60,...,fulladdress_unique_count_for_ssn_lastname_30,fulladdress_unique_count_for_ssn_fulladdress_30,fulladdress_unique_count_for_ssn_30,fulladdress_unique_count_for_ssn_zip5_30,fulladdress_unique_count_for_ssn_name_dob_30,fulladdress_count_0_by_14,fulladdress_unique_count_for_name_fulladdress_30,fulladdress_unique_count_for_name_dob_30,name_dob_count_30,ssn_name_dob_count_30
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,1.064624,1.048903,1.449296,1.046097,1.048207,29.491030,1.044485,1.038698,1.038636,1.038559,...,1.036291,1.036240,1.036240,1.036240,1.038982,13.866195,1.035916,1.038716,1.046295,1.045940
std,0.633831,0.499029,0.861622,0.496315,0.507682,2.831175,0.617953,0.613817,0.613757,0.613686,...,0.608074,0.608032,0.608032,0.608032,0.610242,1.015231,0.607661,0.609942,0.496914,0.496147
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.304348,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.636364,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,2.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000
max,30.000000,34.000000,32.000000,34.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,14.000000,30.000000,30.000000,34.000000,34.000000


In [12]:
X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()

In [13]:
# push in any outlier values
cols = X.columns
X.loc[:,cols] = X[cols].clip(upper=Clip)
X.loc[:,cols] = X[cols].clip(lower=-1*Clip)
# X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
X.describe()

,fulladdress_count_30,ssn_firstname_count_30,homephone_count_3,ssn_dob_count_30,fulladdress_homephone_count_30,fulladdress_count_0_by_30,fulladdress_unique_count_for_dob_homephone_60,fulladdress_unique_count_for_ssn_name_60,fulladdress_unique_count_for_ssn_lastname_60,fulladdress_unique_count_for_ssn_60,...,fulladdress_unique_count_for_ssn_lastname_30,fulladdress_unique_count_for_ssn_fulladdress_30,fulladdress_unique_count_for_ssn_30,fulladdress_unique_count_for_ssn_zip5_30,fulladdress_unique_count_for_ssn_name_dob_30,fulladdress_count_0_by_14,fulladdress_unique_count_for_name_fulladdress_30,fulladdress_unique_count_for_name_dob_30,name_dob_count_30,ssn_name_dob_count_30
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,-0.018176,-0.017457,-0.005814,-0.017649,-0.018020,-8.709183e-14,-0.019380,-0.019726,-0.019730,-0.019736,...,-0.019873,-0.019877,-0.019877,-0.019877,-0.019712,0.002262,-0.019904,-0.019734,-0.017617,-0.017661
std,0.631721,0.579481,0.910935,0.571302,0.576299,1.000000e+00,0.591268,0.579574,0.579416,0.579228,...,0.572438,0.572323,0.572323,0.572323,0.578313,0.975046,0.571322,0.577521,0.572884,0.570790
min,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955,-9.955826e+00,-0.071988,-0.063045,-0.062950,-0.062832,...,-0.059682,-0.059602,-0.059602,-0.059602,-0.063880,-10.000000,-0.059105,-0.063475,-0.093165,-0.092594
25%,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955,1.797736e-01,-0.071988,-0.063045,-0.062950,-0.062832,...,-0.059682,-0.059602,-0.059602,-0.059602,-0.063880,0.131798,-0.059105,-0.063475,-0.093165,-0.092594
50%,-0.101958,-0.097996,-0.521454,-0.092879,-0.094955,1.797736e-01,-0.071988,-0.063045,-0.062950,-0.062832,...,-0.059682,-0.059602,-0.059602,-0.059602,-0.063880,0.131798,-0.059105,-0.063475,-0.093165,-0.092594
75%,-0.101958,-0.097996,0.639148,-0.092879,-0.094955,1.797736e-01,-0.071988,-0.063045,-0.062950,-0.062832,...,-0.059682,-0.059602,-0.059602,-0.059602,-0.063880,0.131798,-0.059105,-0.063475,-0.093165,-0.092594
max,10.000000,10.000000,10.000000,10.000000,10.000000,1.797736e-01,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,0.131798,10.000000,10.000000,10.000000,10.000000


In [14]:
# separate data into modeling (traintest) and out of time
X_trntst = X[0:833507]
Y_trntst = Y_save[0:833507]
X_oot = X[833507:]
Y_oot = Y_save[833507:]

In [15]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [16]:
%%time
# Logistic regression
log = pd.DataFrame(columns=['num_vars','penalty','solver','c','train','test','oot'])
number_var = []
penalty_num = []
solver_type = []
c_value = []
train_value = []
test_value = []
oot_value = []
for num_vars in [5,10,15,20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for penalty in ['l1','l2','elasticnet']:
        for solver in ['lbfgs','saga']:
            for c in [0.1,0.6,1,5]:
                try:
                    for niter in range(nitermax):    
                        X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                        model = LogisticRegression(penalty=penalty,solver=solver,C=c)

                        X_oot = X_oot_orig.copy()
                        X_trn_save = X_trn.copy()
                        Y_trn_save = Y_trn.copy()

                        model.fit(X_trn, Y_trn.values.ravel())   
                        predictions = model.predict_proba(X_trn_save)[:,1]
                        X_trn['predicted'] = predictions
                        X_trn['Fraud'] = Y_trn_save['Fraud']
                        topRows = int(round(X_trn.shape[0]*0.03))
                        temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_tst)[:,1]
                        X_tst['predicted']=predictions
                        X_tst['Fraud'] = Y_tst['Fraud']
                        topRows = int(round(X_tst.shape[0]*0.03))
                        temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_oot_new)[:,1]
                        X_oot['predicted']=predictions
                        X_oot['Fraud'] = Y_oot['Fraud']
                        topRows = int(round(X_oot.shape[0]*0.03))
                        temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                    number_var.append(num_vars)
                    penalty_num.append(penalty)
                    solver_type.append(solver)
                    c_value.append(c)
                    train_value.append(round(FDR3.mean()[0]*100,2))
                    test_value.append(round(FDR3.mean()[1]*100,2))
                    oot_value.append(round(FDR3.mean()[2]*100,2))
                    print("finished!")
                except:
                    print('bad_model')

good
bad_model
bad_model
bad_model
bad_model
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
good
bad_model
bad_model
bad_model
bad_model


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!


...

finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


finished!
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
bad_model
CPU times: user 1h 16min 30s, sys: 43 s, total: 1h 17min 13s
Wall time: 58min 6s


In [18]:
logm = pd.DataFrame(data={'num_vars':number_var,'penalty':penalty_num,'solver':solver_type,'c':c_value,'train':train_value,'test':test_value,'oot':oot_value})

In [19]:
logm.to_excel("ligstic.xlsx")

## Models. You can comment out any of these cells and just explore one model type. You can also just rerun that single cell multiple times as you explore different model hyperparameters.

In [20]:
%%time
# Single DT
number_var = []
criterion_type = []
splitter_type = []
max_depth_num= []
min_samples_leaf_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [5,10,15,20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for criterion in ['gini','entropy']:
        for splitter in ['best','random']:
            for max_depth in [10,20,None]:
                for min_samples_leaf in [1,20,30,40]:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = DecisionTreeClassifier(criterion=criterion,splitter=splitter,max_depth=max_depth,min_samples_leaf=min_samples_leaf)

                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                        number_var.append(num_vars)
                        criterion_type.append(criterion)
                        splitter_type.append(splitter)
                        max_depth_num.append(max_depth)
                        min_samples_leaf_num.append(min_samples_leaf)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model!")

good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
good
finished!
finished!
finis

In [21]:
dt = pd.DataFrame(data={'num_vars':number_var,'criterion':criterion_type,'splitter':splitter_type,'max_depth':max_depth_num,'min_samples_leaf':min_samples_leaf_num,'train':train_value,'test':test_value,'oot':oot_value})

In [22]:
dt.to_excel("decision_tree.xlsx")

In [23]:
%%time
# RF
number_var = []
criterion_type = []
n_estimators_num = []
max_depth_num= []
max_features_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for n_estimators in [10,50,100,150]:
        for max_depth in [10,20,30]:
            for cirterion in ['gini','entropy']:
                for max_features in ["auto","log2"]:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,criterion=criterion,max_features=max_features)
                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                            
                        number_var.append(num_vars)
                        criterion_type.append(criterion)
                        n_estimators_num.append(n_estimators)
                        max_depth_num.append(max_depth)
                        max_features_num.append(max_features)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model!")

good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
CPU times: user 1h 29min 16s, 

In [24]:
rf = pd.DataFrame(data={'num_vars':number_var,'criterion':criterion_type,'n_estimators':n_estimators_num,'max_depth':max_depth_num,'max_features':max_features_num,'train':train_value,'test':test_value,'oot':oot_value})

In [25]:
rf.to_excel('rf.xlsx')

In [26]:
%%time
# BT
number_var = []
learning_rate_num = []
n_estimators_num = []
max_depth_num= []
subsample_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for n_estimators in [100,200,500,1000]:
        for max_depth in [3,4,5]:
            for learning_rate in [0.001,0.01,0.02,0.05,0.1]:
                for subsample in [0.7,0.8,1]:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = lgb.LGBMClassifier(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate,subsample=subsample)

                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                            
                        number_var.append(num_vars)
                        learning_rate_num.append(learning_rate)
                        n_estimators_num.append(n_estimators)
                        max_depth_num.append(max_depth)
                        subsample_num.append(subsample)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model")

good
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finis

In [27]:
bt = pd.DataFrame(data={'num_vars':number_var,'learning_rate':learning_rate_num,'n_estimators':n_estimators_num,'max_depth':max_depth_num,'subsample':subsample_num,'train':train_value,'test':test_value,'oot':oot_value})

In [28]:
bt.to_excel("bt.xlsx")

In [29]:
%%time
# NN
number_var = []
hidden_layer_size_num = []
max_iter_num = []
learning_rate_type = []
train_value = []
test_value = []
oot_value = []
for num_vars in [20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for hidden_layer_sizes in [(5),(10),(20)]:
        for max_iter in [20,50,100]:
            for learning_rate in ['constant', 'invscaling', 'adaptive']:
                    try:
                        for niter in range(nitermax):    
                            X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                            model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,max_iter=max_iter,learning_rate=learning_rate)

                            X_oot = X_oot_orig.copy()
                            X_trn_save = X_trn.copy()
                            Y_trn_save = Y_trn.copy()

                            model.fit(X_trn, Y_trn.values.ravel())   

                            predictions = model.predict_proba(X_trn_save)[:,1]
                            X_trn['predicted'] = predictions
                            X_trn['Fraud'] = Y_trn_save['Fraud']
                            topRows = int(round(X_trn.shape[0]*0.03))
                            temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_tst)[:,1]
                            X_tst['predicted']=predictions
                            X_tst['Fraud'] = Y_tst['Fraud']
                            topRows = int(round(X_tst.shape[0]*0.03))
                            temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                            predictions = model.predict_proba(X_oot_new)[:,1]
                            X_oot['predicted']=predictions
                            X_oot['Fraud'] = Y_oot['Fraud']
                            topRows = int(round(X_oot.shape[0]*0.03))
                            temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                            needed = temp.loc[:,'Fraud']
                            FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
                            
                        number_var.append(num_vars)
                        hidden_layer_size_num.append(hidden_layer_sizes)
                        max_iter_num.append(max_iter)
                        learning_rate_type.append(learning_rate)
                        train_value.append(round(FDR3.mean()[0]*100,2))
                        test_value.append(round(FDR3.mean()[1]*100,2))
                        oot_value.append(round(FDR3.mean()[2]*100,2))
                        print("finished!")
                    except:
                        print("bad model")

good


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


finished!
finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


finished!
finished!
finished!
finished!
finished!
finished!
finished!


...

finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


finished!


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


finished!
finished!
finished!
finished!
finished!
finished!
finished!
finished!
CPU times: user 2h 38min 46s, sys: 2min 6s, total: 2h 40min 53s
Wall time: 1h 20s


In [30]:
nn = pd.DataFrame(data={'num_vars':number_var,'hidden_layer_size':hidden_layer_size_num,'max_iter':max_iter_num,'learning_rate':learning_rate_type,'train':train_value,'test':test_value,'oot':oot_value})

In [31]:
nn.to_excel("nn.xlsx")

In [16]:
%%time
# XGBoost
from xgboost import XGBClassifier
number_var = []
learning_rate_num = []
n_estimators_num = []
max_depth_num = []
train_value = []
test_value = []
oot_value = []
for num_vars in [20,25]:
    X_trntst_new = X_trntst.iloc[:,:num_vars+1]
    X_oot_new = X_oot_orig.iloc[:,:num_vars+1]
    print('good')
    for n_estimators in [80,100,150,200]:
        for learning_rate in [0.05,0.1,0.3,0.4]:
            for max_depth in [3,4,5]:
                try:
                    for niter in range(nitermax):    
                        X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_new, Y_trntst, test_size = .3)

                        model = XGBClassifier(seed=47)

                        X_oot = X_oot_orig.copy()
                        X_trn_save = X_trn.copy()
                        Y_trn_save = Y_trn.copy()

                        model.fit(X_trn, Y_trn.values.ravel())   

                        predictions = model.predict_proba(X_trn_save)[:,1]
                        X_trn['predicted'] = predictions
                        X_trn['Fraud'] = Y_trn_save['Fraud']
                        topRows = int(round(X_trn.shape[0]*0.03))
                        temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_tst)[:,1]
                        X_tst['predicted']=predictions
                        X_tst['Fraud'] = Y_tst['Fraud']
                        topRows = int(round(X_tst.shape[0]*0.03))
                        temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

                        predictions = model.predict_proba(X_oot_new)[:,1]
                        X_oot['predicted']=predictions
                        X_oot['Fraud'] = Y_oot['Fraud']
                        topRows = int(round(X_oot.shape[0]*0.03))
                        temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
                        needed = temp.loc[:,'Fraud']
                        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])

                    number_var.append(num_vars)
                    learning_rate_num.append(learning_rate)
                    n_estimators_num.append(n_estimators)
                    max_depth_num.append(max_depth)
                    train_value.append(round(FDR3.mean()[0]*100,2))
                    test_value.append(round(FDR3.mean()[1]*100,2))
                    oot_value.append(round(FDR3.mean()[2]*100,2))
                    print("finished!")
                except:
                    print("bad model")

good


/Users/zhiyuzhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:19:55] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:20:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:20:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[19:20:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

[19:26:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[19:26:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:26:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:26:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

[19:32:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:32:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[19:33:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:33:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

...

[19:51:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[19:52:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

finished!
[19:59:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:59:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:00:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[20:00:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation me

[20:07:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[20:08:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:08:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:08:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

[20:15:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:15:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[20:15:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:16:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used 

finished!
[20:39:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:39:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:40:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
finished!
[20:40:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation me

In [ ]:
xgboost = pd.DataFrame(data={'num_vars':number_var,'n_estimators':n_estimators_num,'learning_rate':learning_rate_num,'max_depth':max_depth_num,'train':train_value,'test':test_value,'oot':oot_value})

In [ ]:
xgboost.to_excel('xgboost.xlsx')

In [19]:
print('duration: ', datetime.now() - start_time)

duration:  0:03:48.900114
